In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import numpy as np
import math
import pandas as pd
from collections import Counter
import numpy as np
import unicodedata
import pickle
Stopwords = set(stopwords.words('english'))
ps=PorterStemmer()

In [2]:
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z\s]')
    text_returned = re.sub(regex,' ',text)
    return text_returned
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')


In [3]:
with open('pickles/posting_list.pkl','rb') as file:
    tf=pickle.load(file)
    file.close()
    
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_len.pkl','rb') as file:
    doc_len=pickle.load(file)
    file.close()

In [4]:
k=0
Ld=doc_len
N=len(file_idx)
for i in Ld:
    k+=Ld[i]
Lavg=k/N

In [5]:
def IDF(q):
    DF1=0
    if q in DF:
        DF1=DF[q]
    ans=math.log((N-DF1+0.5)/(DF1+0.5))
    return ans
k=1.2
b=0.75

In [6]:
def bm25(query,counter=10):
    query=strip_accents(query)
    score={}
    for i in range(len(file_idx)):
        score[i]=0
    q = remove_special_characters(query)
    words = word_tokenize(q)
    words = [word.lower() for word in words]
    words=[ps.stem(word) for word in words]
    words=[word for word in words if word not in Stopwords]
    for i in range(len(file_idx)):
        score[i]=0
        for qi in words:
            TF=0
            if qi in tf:
                if i in tf[qi]:
                    TF=tf[qi][i]
            idf=IDF(qi)
            ans=idf*(k+1)*TF/(TF+k*(1-b+b*(Ld[i]/Lavg)))
            score[i]+=ans
    score=sorted(score.items(),key=lambda item: item[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index

In [7]:
data=pd.read_csv('../English.csv')

In [8]:
verse_index=bm25('importance of bhagavad gita')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
183,4,42,तस्मादज्ञानसम्भूतं हृत्स्थं ज्ञानासिनात्मनः ।छ...,The yoga system instructed in this chapter is ...
81,2,54,अर्जुन उवाचस्थितप्रज्ञस्य का भाषा समाधिस्थस्य ...,"As there are symptoms for each and every man, ..."
583,18,64,सर्वगुह्यतमं भूय: श‍ृणु मे परमं वच: ।इष्टोऽसि ...,The Lord has given Arjuna knowledge that is co...
591,18,74,इत्यहं वासुदेवस्य पार्थस्य च महात्मन: ।संवादमि...,"In the beginning of Bhagavad-gītā, Dhṛtarāṣṭra..."
410,11,52,श्रीभगवानुवाचसुदुर्दर्शमिदं रूपं दृष्टवानसि यन...,In the forty-eighth verse of this chapter Lord...
0,1,1,धृतराष्ट्र उवाचधर्मक्षेत्रे कुरुक्षेत्रे समवेत...,Bhagavad-gītā is the widely read theistic scie...
306,9,3,अश्रद्दधाना: पुरुषा धर्मस्यास्य परन्तप ।अप्राप...,The faithless cannot accomplish this process o...
184,5,1,अर्जुन उवाचसन्न्यासं कर्मणां कृष्ण पुनर्योगं च...,"In this Fifth Chapter of the Bhagavad-gītā, th..."
41,2,12,न त्वेवाहं जातु नासं न त्वं नेमे जनाधिपाः ।न च...,In the Vedas – in the Kaṭha Upaniṣad as well a...
371,10,38,दण्डो दमयतामस्मि नीतिरस्मि जिगीषताम् ।मौनं चैव...,"There are many suppressing agents, of which th..."
